## TextCNN

In [1]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

class TextCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        # Embedding part can try multichannel as same as origin paper
        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        convs = []
        for kernel_size in [3, 4, 5]:
            c = Conv1D(128, kernel_size, activation='relu')(embedding)
            c = GlobalMaxPooling1D()(c)
            convs.append(c)
        x = Concatenate()(convs)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

Using TensorFlow backend.


In [2]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 100
epochs = 10

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

result = model.predict(x_test)
result[result>=0.5] = 1
result[result<0.5] = 0
acc = accuracy_score(result, y_test)
print('Test data accuracy is ', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)...
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Instructions for updating:
Colocations handled automatically by placer.
Train...
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 73s 3ms/step - loss: 0.3733 - acc: 0.8269 - val_loss: 0.3171 - val_acc: 0.8648
Epoch 2/10
25000/25000 [==============================] - 75s 3ms/step - loss: 0.1947 - acc: 0.9236 - val_loss: 0.2440 - val_acc: 0.9023
Epoch 3/10
25000/25000 [==============================] - 77s 3ms/step - loss: 0.1061 - acc: 0.9654 - val_loss: 0.2685 - val_acc: 0.8967
Epoch 4/10
25000/25000 [==============================] - 79s 3ms/step - loss: 0.0482 - acc: 0.9878 - val_loss: 0.3108 - val_acc: 0.8932
Epoch 5/10
25000/25000 [========

In [3]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, result)
print("auc is ", auc)

auc is  0.89188
